<a href="https://colab.research.google.com/github/deemoo0/PG-Project/blob/main/analysiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import Libraries


In [2]:
!pip install gensim
!pip install nltk
!pip install  tensorflow
!pip install  keras
!pip install  matplotlib

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


Load Dataset


In [5]:
df = pd.read_csv(
    "/content/drive/MyDrive/training.1600000.processed.noemoticon.csv",
    encoding="latin-1",
    header=None
)

df.columns = ["sentiment", "id", "date", "query", "user", "text"]
df = df[["sentiment", "text"]]

df.head()


,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [6]:
display(df.head())

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [7]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")

stop_words = set(stopwords.words("english"))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


3.NLP Setup

In [8]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    tokens = word_tokenize(text)
    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word not in stop_words and len(word) > 2
    ]
    return " ".join(tokens)


4.Apply Cleaning

In [9]:
df_sample = df.sample(80000, random_state=42)
df_sample["clean_text"] = df_sample["text"].apply(clean_text)

df_sample.head()



,sentiment,text,clean_text
541200,0,@chrishasboobs AHHH I HOPE YOUR OK!!!,ahhh hope
750,0,"@misstoriblack cool , i have no tweet apps fo...",cool tweet apps razr
766711,0,@TiannaChaos i know just family drama. its la...,know family drama lamehey next time hang kim g...
285055,0,School email won't open and I have geography ...,school email wont open geography stuff revise ...
705995,0,upper airways problem,upper airway problem


In [10]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

5.demand mapping

In [11]:
def map_demand(sentiment):
    if sentiment == 4:
        return "High"
    else:
        return "Low"

df_sample["demand"] = df_sample["sentiment"].apply(map_demand)



In [12]:
le = LabelEncoder()
y = le.fit_transform(df_sample["demand"])


In [13]:
df_sample["demand"].value_counts(normalize=True)


,proportion
demand,
High,0.500012
Low,0.499987


6.Tokenization for Word3Vec

In [14]:
from gensim.models import Word2Vec
sentences = [text.split() for text in df_sample["clean_text"]]


7.Train Word2Vec Model

In [15]:
w2v_model = Word2Vec(
    sentences=sentences,
    vector_size=200,
    window=8,
    min_count=3,
    workers=4,
    sg=1,
    epochs=20
)


8.Check Word Embedding

In [16]:
w2v_model.wv.most_similar("good")


[('goooooood', 0.574455976486206),
 ('canberra', 0.5565109252929688),
 ('guten', 0.5563056468963623),
 ('drift', 0.5561344623565674),
 ('twitterville', 0.5519238710403442),
 ('grump', 0.547402024269104),
 ('frnds', 0.5465630292892456),
 ('uneventful', 0.5454241633415222),
 ('marvelous', 0.5407905578613281),
 ('kinsale', 0.538668692111969)]

9.build sequence Matrix

In [17]:
MAX_LEN = 120
EMBEDDING_DIM = 200

def text_to_sequence(text):
    words = text.split()
    vectors = [
        w2v_model.wv[word]
        for word in words
        if word in w2v_model.wv
    ][:MAX_LEN]

    if len(vectors) < MAX_LEN:
        vectors += [np.zeros(EMBEDDING_DIM)] * (MAX_LEN - len(vectors))

    return np.array(vectors)




In [ ]:
X = np.array([text_to_sequence(text) for text in df_sample["clean_text"]])
X.shape


In [3]:
df_sample["demand"].value_counts(normalize=True)


NameError: name 'df_sample' is not defined

In [58]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y),
    y=y
)

class_weight_dict = dict(enumerate(class_weights))


10.encode Labels

In [60]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df_sample["demand"])

X.shape, y.shape


NameError: name 'X' is not defined

In [ ]:
df_sample['demand'] = df_sample['sentiment'].apply(map_demand)

In [ ]:
display(df_sample.head())

11.train test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape


12.import deep learning libs

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.utils import to_categorical


cell13
one hot encode output

In [ ]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

y_train_cat.shape


14.build bilstm model

In [ ]:
model = Sequential()

model.add(
    Bidirectional(
        LSTM(128, return_sequences=True),
        input_shape=(MAX_LEN, EMBEDDING_DIM)
    )
)

model.add(Dropout(0.3))

model.add(
    Bidirectional(
        LSTM(64, return_sequences=False)
    )
)

model.add(Dropout(0.3))

model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(2, activation="softmax"))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


15 .train the model

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

callbacks = [
    EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2)
]

history = model.fit(
    X_train,
    y_train_cat,
    validation_data=(X_test, y_test_cat),
    epochs=25,
    batch_size=128,
    class_weight=class_weight_dict,
    callbacks=callbacks
)



16.model evaluation

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test_cat)
accuracy


17.accuracy gragh

In [ ]:
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("BiLSTM Demand Prediction Accuracy")
plt.show()
